In [1]:
import numpy as np
import sympy as sym

L = 1
T0 = 1
A_tilde = 4e-4
a_tilde = 4e-5
q_tilde = 0.0
K_tilde = 50e3
k_tilde = 500
Pe_tilde = 0.0
pe_tilde = 50
m = 0.5
n = 0
rho = 1.05
K_r = 2.4160

x, t = sym.symbols('x, t')
K_sym, A_sym, A0_sym, q_sym = sym.symbols('K, A, A0, q')

A = A_tilde + a_tilde * sym.sin(2 * np.pi / L * x) * sym.cos(2 * np.pi / T0 * t)

q = q_tilde - (a_tilde * L / T0) * sym.cos(2 * np.pi / L * x) * sym.sin(2 * np.pi / T0 * t)

#K = K_tilde + k_tilde * sym.sin(2 * np.pi / L * x)
K = K_tilde

#Pe = Pe_tilde + pe_tilde * sym.sin(2 * np.pi / L * x)
Pe = Pe_tilde

#A0 = A_tilde + a_tilde * sym.sin(2 * np.pi / L * x)
A0 = A_tilde

#Psi = K_sym * ((A_sym / A0_sym) ** m - (A_sym / A0_sym) ** n)
Psi = K * ((A_sym / A0) ** m - (A_sym / A0) ** n)

U = sym.Matrix([A, q])

In [2]:
from sympy import Number

def round_expr(expr, num_digits):
    return expr.xreplace({n : round(n, num_digits) for n in expr.atoms(Number)})

In [3]:
H = sym.Matrix([[0, 1], [A_sym / rho * sym.diff(Psi, A_sym) - (q_sym / A_sym) ** 2 , 2 * q_sym / A_sym]])
#H = H.subs({A0_sym: A0, K_sym: K})
#H = sym.simplify(H)
H = round_expr(H, 5)
H

Matrix([
[                               0,     1],
[1190476.19048*A**0.5 - q**2/A**2, 2*q/A]])

In [4]:
B = sym.Matrix([0, K_r * q_sym / A_sym + A_sym / rho * (sym.diff(Psi, A0_sym) * sym.diff(A0, x) + sym.diff(Psi, K_sym) * sym.diff(K, x))])
B = B.subs({A0_sym: A0, K_sym: K})
#B = sym.simplify(B)
B = round_expr(B, 5)
B

Matrix([
[        0],
[2.416*q/A]])

In [5]:
C1 = sym.integrate(A_sym / rho * sym.diff(Psi, A_sym), (A_sym, A0_sym, A_sym))
C1 = C1.subs({A0_sym: A0, K_sym: K})
C1 = sym.simplify(C1)
C1 = round_expr(C1, 5)
C1

793650.79365*A**1.5 - 6.34921

In [6]:
F = sym.Matrix([q_sym, q_sym ** 2/ A_sym + C1])
F = F.subs({A0_sym: A0, K_sym: K})
F = sym.simplify(F)
F = round_expr(F, 5)
F

Matrix([
[                                     q],
[793650.79365*A**1.5 - 6.34921 + q**2/A]])

In [7]:
S = B - sym.Matrix([0, (sym.diff(C1, A0_sym) * sym.diff(A0, x) + sym.diff(C1, K_sym) * sym.diff(K, x))])
S = S.subs({A0_sym: A0, K_sym: K})
S = round_expr(S, 5)
S

Matrix([
[        0],
[2.416*q/A]])

In [8]:
F = F.subs({A_sym: A, q_sym: q})
dF_dx = sym.diff(F, x)
dF_dx = round_expr(dF_dx, 5)
dF_dx

Matrix([
[                                             0.00025*sin(6.28319*t)*sin(6.28319*x)],
[5.98399*(0.1*sin(6.28319*x)*cos(6.28319*t) + 1)**0.5*cos(6.28319*t)*cos(6.28319*x)]])

In [9]:
dU_dt = sym.diff(U, t)
dU_dt = round_expr(dU_dt, 5)
dU_dt

Matrix([
[-0.00025*sin(6.28319*t)*sin(6.28319*x)],
[-0.00025*cos(6.28319*t)*cos(6.28319*x)]])

In [10]:
S = S.subs({A_sym: A, q_sym: q})
S = round_expr(S, 5)
S

Matrix([
[                                                                                    0],
[-0.0001*sin(6.28319*t)*cos(6.28319*x)/(4.0e-5*sin(6.28319*x)*cos(6.28319*t) + 0.0004)]])

In [11]:
f = dU_dt + dF_dx + S
f = round_expr(f, 5)
f

Matrix([
[                                                                                                                                                                                                                0],
[5.98399*(0.1*sin(6.28319*x)*cos(6.28319*t) + 1)**0.5*cos(6.28319*t)*cos(6.28319*x) - 0.00025*cos(6.28319*t)*cos(6.28319*x) - 0.0001*sin(6.28319*t)*cos(6.28319*x)/(4.0e-5*sin(6.28319*x)*cos(6.28319*t) + 0.0004)]])

In [12]:
f_code = sym.printing.ccode(f)
print('f =', f_code)

f = /* Not supported in C: */
/* ImmutableDenseMatrix */
Matrix([
[                                                                                                                                                                                                                0],
[5.98399*(0.1*sin(6.28319*x)*cos(6.28319*t) + 1)**0.5*cos(6.28319*t)*cos(6.28319*x) - 0.00025*cos(6.28319*t)*cos(6.28319*x) - 0.0001*sin(6.28319*t)*cos(6.28319*x)/(4.0e-5*sin(6.28319*x)*cos(6.28319*t) + 0.0004)]])
